In [1]:
import pandas as pd
import pyodbc as db
import db_config as config
import numpy as np
import time

#### Connect to the Database

In [19]:
conn = db.connect(DRIVER='SQL Server',
                 SERVER = config.server_name,
                 UID = config.user,
                 PWD=config.pwd,
                 DATABASE=config.database_name)

In [20]:
conn

#### Read the Tags from the excel sheet and modify data

In [3]:
monitor = pd.read_excel('MonitoringData.xlsx')
monitor2= monitor.dropna(subset='Address').reset_index().drop(columns='index')
monitor2['Address'] = [int(i) for i in monitor2['Address']]

In [4]:
cols = monitor2.columns.tolist()
for col in cols:
    monitor2[col] = [str(i) for i in monitor2[col]]

In [5]:
monitor2 = monitor2.replace('nan', 'NA')

#### Custom functions to create table, Write, read and delete tags

In [6]:
def createTable(db_name, tablename, columns, conn):
    insertCMD = f'''CREATE TABLE "{tablename}" ({columns});'''
    cur = conn.cursor()
    try:
        cur.execute(insertCMD)
        conn.commit()
        print(f"Table {tablename} was created in DB {db_name}")
    except(Exception, db.DatabaseError) as error:
        print(error)

In [7]:
def writeValues(metrics, conn, table):
    try:
        cur = conn.cursor()
    except (Exception, db.DatabaseError) as error:
        print(error)
    keys = list(metrics.keys())
    values = tuple(metrics.values())
    cols = '"' + ('","').join(keys) + '"'
    s_lens = "?,"*len(keys)
    s = s_lens.split(",")
    s = (",").join(s[:-1])
    insertQ = f""" INSERT INTO {table} ({cols})
                    VALUES({s})"""
    try:
        cur.execute(insertQ, values)
        conn.commit()
        print(f'Values Inserted: {values}')
    except (Exception, db.DatabaseError) as error:
        print(error)

In [21]:
def getData(tablename, conn):
    qu = f'select * from "{tablename}"'
    alldata = pd.read_sql_query(qu, conn)
    return alldata

In [9]:
def delData(tablename, conn):
    try:
        cur = conn.cursor()
        q = f"delete from {tablename};"
        cur.execute(q)
        l = getData(tablename, conn)
        if len(l['Address'].tolist()) == 0:
            print("Delete Succesful")
    except (Exception, db.DatabaseError) as error:
        print(error)

#### Write the tag info and test if all info is available

In [10]:
data = monitor2.to_dict('records')
for metrics in data:
    try:
        writeValues(metrics, conn, "PoC_SP_MonitoringTags")
    except pyodbc.Error as pe:
        print("Error:", pe)
        if pe.args[0] == "08S01":  # Communication error.
            # Nuke the connection and retry.
            try:
                conn.close()
                print("Connection Ended")
            except:
                pass
            continue

Values Inserted: ('207', 'ActualSetpointManualHI', '0.1 ml/h', 'The actual setpoint used in operating mode "Manual".')
Values Inserted: ('208', 'ActualSetpointManualLO', 'NA', 'Can be set via SetpointManual (register 00106-00107) or via the pump HMI.')
Values Inserted: ('209', 'ActualPulseVolumeHI', '1 nl', 'The actual pulse volume used in operating mode "Pulse".')
Values Inserted: ('210', 'ActualPulseVolumeLO', 'NA', 'Can be set via SetPulseVolume (register 00108-00109) or via the pump HMI.')
Values Inserted: ('211', 'ActualBatchDosingVolumeHI', '0.001 ml', 'Can be set via SetBatchDosingVolume (register 00110-00111) or via the pump')
Values Inserted: ('212', 'ActualBatchDosingVolumeLO', 'NA', 'HMI.')
Values Inserted: ('213', 'ActualBatchDosingTimeHI', '0.1 s', 'The actual batch dosing time used in operating mode "Batch".')
Values Inserted: ('214', 'ActualBatchDosingTimeLO', 'NA', 'Can be set via SetBatchDosingTime (register 00112-00113) or via the pump HMI.')
Values Inserted: ('215', 

In [19]:
getData("PoC_SP_MonitoringTags", conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Address,Register name,Scale,Description
0,207,ActualSetpointManualHI,0.1 ml/h,"The actual setpoint used in operating mode ""Ma..."
1,208,ActualSetpointManualLO,NA,Can be set via SetpointManual (register 00106-...
2,209,ActualPulseVolumeHI,1 nl,The actual pulse volume used in operating mode...
3,210,ActualPulseVolumeLO,NA,Can be set via SetPulseVolume (register 00108-...
4,211,ActualBatchDosingVolumeHI,0.001 ml,Can be set via SetBatchDosingVolume (register ...
...,...,...,...,...
81,324,OperatingHoursLO,NA,when the pump is dosing and when it is not dos...
82,325,StrokeCounterHI,-,Counts the number of strokes (non-resettable).
83,326,StrokeCounterLO,NA,NA
84,327,TimeToNextDosingHI,1 s,Time before the next dosing takes place (only ...


In [13]:
columns = ""
for col in monitor2['Address'].tolist():
    if columns == "":
        columns = f'"{col}" text'
    else:
        columns = f'{columns}, "{col}" text'

In [16]:
createTable(config.database_name, 'PoC_SP_Metrics', columns, conn)

Table PoC_SP_Metrics was created in DB budig-bb-pltsql-05-d


In [20]:
data

[{'Address': '207',
  'Register name': 'ActualSetpointManualHI',
  'Scale': '0.1 ml/h',
  'Description': 'The actual setpoint used in operating mode "Manual".'},
 {'Address': '208',
  'Register name': 'ActualSetpointManualLO',
  'Scale': 'NA',
  'Description': 'Can be set via SetpointManual (register 00106-00107) or via the pump HMI.'},
 {'Address': '209',
  'Register name': 'ActualPulseVolumeHI',
  'Scale': '1 nl',
  'Description': 'The actual pulse volume used in operating mode "Pulse".'},
 {'Address': '210',
  'Register name': 'ActualPulseVolumeLO',
  'Scale': 'NA',
  'Description': 'Can be set via SetPulseVolume (register 00108-00109) or via the pump HMI.'},
 {'Address': '211',
  'Register name': 'ActualBatchDosingVolumeHI',
  'Scale': '0.001 ml',
  'Description': 'Can be set via SetBatchDosingVolume (register 00110-00111) or via the pump'},
 {'Address': '212',
  'Register name': 'ActualBatchDosingVolumeLO',
  'Scale': 'NA',
  'Description': 'HMI.'},
 {'Address': '213',
  'Register

In [1]:
import dash
from dash.dependencies import Input, Output, State
from dash import dash_table
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc

In [6]:
def body():
    return html.Div(
                    children=
                    [
                        dbc.Card(
                            dbc.CardBody(
                                id='high-risk',
                                children=[
                                    html.H4("High Risk", className="card-title"),
                                    html.P(
                                        html.Div(
                                        dcc.Graph(id='graph'))
                                    ),
                                ]
                            ),
                            style={"width": "50rem", "height": "30rem"},
                        )
                    ], style={'display': 'inline-block', 'marginLeft': 15}
                )

In [3]:
external_stylesheets = [dbc.themes.MINTY]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.config.suppress_callback_exceptions = True

In [7]:
app.layout = html.Div(
    style={'background': 'white', 'width': '100%', 'height': '100%'},
    id="big-app-container",
    children=[
        body()
    ]
)

In [8]:
if __name__ == "__main__":
    app.run(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [18/Sep/2022 13:19:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 13:19:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 13:19:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 13:19:07] "GET /_favicon.ico?v=2.5.1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 13:19:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Sep/2022 13:19:07] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


In [13]:
%tb

SystemExit: 1

In [2]:
data = {"site": "digitalHUB", "pump": "dda1", "timestamp": "2022-09-19 16:27:30.316819", "metrics": [{"207": "37500"}, {"208": "11"}, {"209": "24104"}, {"210": "1"}, {"211": "8964"}, {"212": "0"}, {"213": "600"}, {"214": "170"}, {"215": "4"}, {"216": "0"}, {"217": "0"}, {"218": "0"}, {"219": "18"}, {"220": "7"}, {"221": "18"}, {"222": "0"}, {"223": "41"}, {"224": "49"}, {"225": "65535"}, {"301": "1"}, {"302": "9464"}, {"303": "0"}, {"304": "0"}, {"305": "0"}, {"306": "0"}, {"307": "10"}, {"308": "0"}, {"309": "0"}, {"310": "0"}, {"311": "0"}, {"312": "4509"}, {"313": "0"}, {"314": "4314"}, {"315": "0"}, {"320": "0"}, {"321": "0"}, {"322": "4"}, {"323": "63116"}, {"324": "0"}, {"325": "5969"}, {"326": "0"}, {"327": "0"}, {"328": "65535"}]}

In [6]:
l = data['metrics']

In [13]:
list(l[0].keys())[0]

'207'

In [14]:
list(l[0].values())[0]

'37500'

In [15]:
cols = [list(i.keys())[0] for i in l]

In [17]:
vals = [list(i.values())[0] for i in l]

In [18]:
vals

['37500',
 '11',
 '24104',
 '1',
 '8964',
 '0',
 '600',
 '170',
 '4',
 '0',
 '0',
 '0',
 '18',
 '7',
 '18',
 '0',
 '41',
 '49',
 '65535',
 '1',
 '9464',
 '0',
 '0',
 '0',
 '0',
 '10',
 '0',
 '0',
 '0',
 '0',
 '4509',
 '0',
 '4314',
 '0',
 '0',
 '0',
 '4',
 '63116',
 '0',
 '5969',
 '0',
 '0',
 '65535']

In [23]:
getData('PoC_SP_MonitoringTags',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Address,Register name,Scale,Description
0,207,ActualSetpointManualHI,0.1 ml/h,"The actual setpoint used in operating mode ""Ma..."
1,208,ActualSetpointManualLO,NA,Can be set via SetpointManual (register 00106-...
2,209,ActualPulseVolumeHI,1 nl,The actual pulse volume used in operating mode...
3,210,ActualPulseVolumeLO,NA,Can be set via SetPulseVolume (register 00108-...
4,211,ActualBatchDosingVolumeHI,0.001 ml,Can be set via SetBatchDosingVolume (register ...
...,...,...,...,...
81,324,OperatingHoursLO,NA,when the pump is dosing and when it is not dos...
82,325,StrokeCounterHI,-,Counts the number of strokes (non-resettable).
83,326,StrokeCounterLO,NA,NA
84,327,TimeToNextDosingHI,1 s,Time before the next dosing takes place (only ...
